# Оценка эффекта для Контрагентов сегмента "Крупнейшие заемщики" (TOP, компании и группы)
  
  
<br>
  
  
# Содержание <a name=content></a>
  
<br>
  
  
- [<span style="color:red">**Версионность**</span>](#versions)
- [<span style="color:red">**Конфигурационные файлы**</span>](#config_files)   
<br>
  
- [<span style="color:brown">**Импорт библиотек + Master Scale + Словарь**</span>](#initial_import)
- [Определение функций](#functions)   
<br>

- [Ключевые кейсы](#key_cases)   
- [**Загрузка финансовой отчетности**](#upload_financials)
  
  
<br>
  
**ЧАСТЬ I   -- Расчет Рейтингов по Контрагентам**
___
  
  
- [**Загрузка данных**](#upload_data)
  
  
- [**Feature Engineering**](#features)
- [Merge StandAlone-score and Group-score](#mrg)
- [Calibration](#clbr)
- [RWA](#rwa)
- [Government Impact](#gov)


<br>
  
  
# Версионность <a name= versions></a>
  
  
[Содержание](#content)

In [1]:
c_version_r = "050_4"
#
c_version_r_top = ""
c_version_r_big = ""
#
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important ; }</style>"))

<br>
  
  
# Конфигурационные файлы <a name= config_files></a>
  
  
[Содержание](#content)

In [ ]:
#
#
#  ----- 1. ФАЙЛ ОСНОВНОЙ КОНФИГУРАЦИОННЫЙ
#
#
___pathr_config_base___ = "_________base_config_file___v1.03.xlsx"
#
#
#  ----- 2. ФАЙЛЫ КОНФИГУРАЦИОННЫЕ МОДЕЛЕЙ и МОДУЛЕЙ
#
#
#
#
#  ----- 3. ФАЙЛЫ ДАННЫХ ФИНАНСОВОЙ ОТЧЕТНОСТИ
#
#
l___info__ifrs_ = ["",""]
l___info__ras_  = ["",""]
#
#
#  ----- 5. ФАЙЛЫ ПРОЧИХ ДАННЫХ
#
#

<br>
  
  
# Импорт библиотек + Master Scale + Словарь <a name=initial_import></a>


[Содержание](#content)

<br>
  
  
# Определение функций <a name= functions></a>
  
  
[Содержание](#content)

## correlation analysis

In [2]:
def Somers_D(xx,yy):
    
    x = st.rankdata(xx);
    y = st.rankdata(yy);
    n2const = y.size*(y.size-1)/2;
    n=np.zeros(np.unique(y).size);
    
    for i in range (0,np.unique(y).size):
        for j in range(0,y.size):
            if (y[j] == np.unique(y)[i]):
                n[i]+=1;
    y_adj = 0;
    
    for ii in range (0,np.unique(y).size):
        y_adj += 0.5*n[ii]*(n[ii]-1);
        
        K = 0;
        
    for kk in range(0,y.size-1):
        for kkk in range(kk+1,y.size):
            K += np.sign(x[kk] - x[kkk])*np.sign(y[kk] - y[kkk]);
            
    return K/(n2const-y_adj);
            


<br>

# Загрузка и обработка данных <a name=upld></a>

- [Содержание](#content)

In [47]:
%%time

l___names = [
    
    'AMPY'
,'APA'
,'AR'
,'BATL'
,'BCEI'
,'BRY'
,'BSM'
,'CDEV'
,'CHK'
,'CLR'
,'CNX'
,'COG'
,'COP'
,'CPE'
,'CRBO'
,'CRC'
,'CRK'
,'DEN'
,'DVN'
,'EGY'
,'EOG'
,'EQT'
,'ESTE'
,'FANG'
,'GDP'
,'HES'
,'LONE'
,'LPI'
,'MCF'
,'MGY'
,'MNRL'
,'MRO'
,'MTDR'
,'MUR'
,'NOG'
,'OAS'
,'OVV'
,'PDCE'
,'PVAC'
,'PXD'
,'REI'
,'RRC'
,'SBOW'
,'SD'
,'SM'
,'SWN'
,'TALO'
,'TPL'
,'VNOM'
,'WLL'
,'WTI'
,'XEC'
,'XOG'
    
]

l___components = [
    ['ticker','тикер']
,['report_date','дата отчетности']
,['PL__Revenue','Выручка']
,['PL__Cost of Revenue','Себестоимость выручки']
,['PL__Gross Profit','Валовая прибыль']
,['PL__Gross margin','Валовая маржа']
,['PL__Other Operating Income','Прочие операционные доходы']
,['PL__Operating Expenses','Операционные расходы']
,['PL__Selling, General & Admin','Коммерческие, общехозяйственные и административные расходы']
,['PL__Research & Development','Исследования и разработки']
,['PL__Depreciation & Amortization','Износ и амортизация']
,['PL__Prov. For Doubful Accounts','Резерв по сомнительным счетам']
,['PL__Other Operating Expenses','Прочие операционные расходы']
,['PL__Operating Income','Операционные доходы']
,['PL__Operating Margin','Маржа операционной деятельности']
,['PL__Non-Operating Income (Loss)','Неоперационные доходы (убытки)']
,['PL__Interest Expense, Net','Процентные расходы, нетто']
,['PL__Interest Expense','Процентные расходы']
,['PL__Interest Income','Процентные доходы']
,['PL__Other Investment Income (Loss)','Прочий инвестиционный доход (убыток)']
,['PL__Foreign Exch Gain (Loss)','Прибыль (убыток) от операций с иностранной валютой']
,['PL__Income (Loss) from Affiliates (Pre Tax)','Прибыль (убыток) от аффилированных компаний (до налогообложения)']
,['PL__Other Non-Op Income (Loss)','Прочий внереализационный доход (убыток)']
,['PL__Pretax Income','Доходы до налогообложения']
,['PL__Income Taxes','Налоги на прибыль']
,['PL__Income (Loss) from Affiliates (After Tax)','Прибыль (убыток) от аффилированных компаний (после налогообложения)']
,['PL__Income (Loss) from Cont Ops','Прибыль (убыток) от основной деятельности']
,['PL__Net Extraordinary Items','Чистые чрезвычайные статьи']
,['PL__Discontinued Operations NI','Прекращенная деятельность NI']
,['PL__XO & Accounting Changes','XO и изменения в бухгалтерском учете']
,['PL__Minority Interest','Доля меньшинства']
,['PL__Net Income','Чистый доход']
,['PL__Preferred Dividends','Привилегированные дивиденды']
,['PL__Other Adjustments','Прочие корректировки']
,['PL__Net Income Avail to Common','Чистая прибыль, приходящаяся на общую долю']
,['PL__Net Income Avail to Common, Adj','Чистая прибыль, приходящаяся на обыкновенные акции, корректировка']
,['PL__Net Abnormal Losses (Gains)','Чистые ненормальные убытки (прибыли)']
,['PL__Net Extraordinary Losses (Gains)','Чистые сверхнормативные убытки (прибыли)']
,['PL__Depreciation','Амортизация']
,['PL__Amortization','Амортизация']
,['PL__EBIT','EBIT']
,['PL__EBITDA','EBITDA']
,['PL__NPAT','NPAT']
,['BS__Cash, Cash Equivalents & STI','Денежные средства, денежные эквиваленты и НТИ']
,['BS__Cash & Equivalents','Денежные средства и эквиваленты']
,['BS__ST Investments','Инвестиции в ЗП']
,['BS__Accounts & Notes Receivable','Дебиторская задолженность по счетам и векселям']
,['BS__Accounts Receivable','Счета к получению']
,['BS__Notes Receivable','Векселя к получению']
,['BS__Inventories','Товарно-материальные запасы']
,['BS__Raw Materials','Сырьевые материалы']
,['BS__Work In Process','Незавершенное производство']
,['BS__Finished Goods','Готовая продукция']
,['BS__Other Inventory','Прочие запасы']
,['BS__Unbilled Revenue','Невыплаченная выручка']
,['BS__Other ST Assets','Прочие активы НТ']
,['BS__Prepaid Expenses','Предоплаченные расходы']
,['BS__Derivative & Hedging Assets','Производные и хеджирующие активы']
,['BS__Assets Heldfor-Sale','Активы, предназначенные для продажи']
,['BS__ST Deferred Tax Assets','Отложенные налоговые активы СТ']
,['BS__Income Taxes Receivable','Налоги на прибыль к получению']
,['BS__Discontinued Operations ST Assets','Прекращенная деятельность Активы СТ']
,['BS__Misc ST Assets','Прочие активы СТ']
,['BS__Total Current Assets','Итого Текущие активы']
,['BS__Property, Plant & Equip, Net','Основные средства, установки и оборудование, нетто']
,['BS__PP&E Gross','Основные средства, нетто']
,['BS__Accumulated Depreciation','Накопленная амортизация']
,['BS__LT Investments & Receivables','Инвестиции и дебиторская задолженность LT']
,['BS__LT Investments','Инвестиции ЛТ']
,['BS__LT Marketable Securities','Рыночные ценные бумаги ЛТ']
,['BS__LT Receivables','Дебиторская задолженность ЛТ']
,['BS__Goodwill & Intangibles','Гудвилл и нематериальные активы']
,['BS__Goodwill','Гудвилл']
,['BS__Other Intangible Assets','Прочие нематериальные активы']
,['BS__Other LT Assets','Прочие активы ЛТ']
,['BS__Prepaid Expense','Предоплаченные расходы']
,['BS__LT Deferred Tax Assets','LT Отложенные налоговые активы']
,['BS__LT Derivative & Hedging Assets','Активы ЛТ Производные и хеджирующие активы']
,['BS__Prepaid Pension Costs','Предоплаченные пенсионные расходы']
,['BS__Discontinued Operations LT Assets','Прекращенная деятельность Активы LT']
,['BS__Investments in Affiliates','Инвестиции в аффилированные компании']
,['BS__Misc LT Assets','Прочие долгосрочные активы']
,['BS__Total Long-Term Assets','Итого долгосрочные активы']
,['BS__Total Assets','Итого активы']
,['BS__Payables & Accruals','Кредиторская задолженность и начисления']
,['BS__Accounts Payable','Кредиторская задолженность']
,['BS__Accruals & Other','Начисления и прочее']
,['BS__Accrued Income Taxes','Начисленные налоги на прибыль']
,['BS__Interest & Dividends Payable','Проценты и дивиденды к уплате']
,['BS__Other Accrued Liabilities','Прочие начисленные обязательства']
,['BS__ST Debt','Долг ЗП']
,['BS__ST Borrowings','Кредиты и займы СТ']
,['BS__ST Capital Leases','Капитальный лизинг ЗП']
,['BS__Current Portion of LT Debt','Текущая часть долгосрочного долга']
,['BS__Other ST Liabilities','Прочие обязательства ЗП']
,['BS__ST Deferred Revenue','ST Отложенные доходы']
,['BS__ST Derivatives & Hedging Liab','СТ Производные инструменты и обязательства по хеджированию']
,['BS__ST Deferred Tax Liabilities','Отложенные налоговые обязательства']
,['BS__Discontinued Operations ST Liab','Обязательства по прекращенной деятельности']
,['BS__Misc ST Liabilities','Прочие обязательства по ЗП']
,['BS__Total Current Liabilities','Итого текущие обязательства']
,['BS__LT Debt','Долг ЛТ']
,['BS__LT Borrowings','Заемные средства ЛТ']
,['BS__LT Capital Leases','Капитальный лизинг LT']
,['BS__Other LT Liabilities','Прочие обязательства ЛТ']
,['BS__Accrued Liabilities','Начисленные обязательства']
,['BS__Pension Liabilities','Пенсионные обязательства']
,['BS__Pensions','Пенсии']
,['BS__Deferred Compensation','Отложенные компенсации']
,['BS__LT Deferred Revenue','LT Отложенные доходы']
,['BS__Deferred Tax Liabilities','Отложенные налоговые обязательства']
,['BS__Derivatives & Hedging','Производные инструменты и хеджирование']
,['BS__Discontinued Operations LT Liab','Прекращенная деятельность Обязательства ЛТ']
,['BS__Misc LT Liabilities','Прочие долгосрочные обязательства']
,['BS__Total Noncurrent Liabilities','Итого долгосрочные обязательства']
,['BS__Total Liabilities','Итого обязательства']
,['BS__Preferred Equity','Привилегированный капитал']
,['BS__Share Capital & APIC','Акционерный капитал и APIC']
,['BS__Common Stock','Обыкновенные акции']
,['BS__Additional Paid in Capital','Дополнительный оплаченный капитал']
,['BS__Other Share Capital','Прочий акционерный капитал']
,['BS__Treasury Stock','Казначейские акции']
,['BS__Retained Earnings','Нераспределенная прибыль']
,['BS__Other Equity','Прочий капитал']
,['BS__Minority / Non Cont. Interest','Миноритарные / неконтр. Доля участия']
,['BS__Total Equity','Итого капитал']
,['BS__Total Liabilities & Equity','Итого обязательства и капитал']
,['BS__Cash and Marketable Securities','Денежные средства и рыночные ценные бумаги']
,['BS__Restricted Cash','Ограниченные денежные средства']
,['BS__Total Debt','Общий долг']
,['BS__Net Debt','Чистый долг']
,['BS__Total Capital','Итого капитал']
,['BS__Line of Credit','Кредитная линия']
,['BS__Line of Credit Available','Доступная кредитная линия']
,['BS__Line of Credit Used','Использованная кредитная линия']
,['CF__Net Income','Чистый доход']
,['CF__Dep & Amort','Деп и амортизация']
,['CF__NonCash Items','Неденежные статьи']
,['CF__StockBased Compensation','Компенсации на основе акций']
,['CF__Deferred Income Taxes','Отложенные налоги на прибыль']
,['CF__Change in Working Capital','Изменение в оборотном капитале']
,['CF__Net Cash From Disc Ops','Чистые денежные средства от операционной деятельности']
,['CF__Cash from Operations','Денежные средства от операционной деятельности']
,['CF__Change in Fixed & Intang','Изменение в основных и внутренних']
,['CF__Capex','Капитальные вложения']
,['CF__Dec in LT Investment','Снижение в долгосрочных инвестициях']
,['CF__Inc in LT Investment','Увеличение в долгосрочных инвестициях']
,['CF__Net Cash From Acq & Div','Чистые денежные средства от приобретения и продажи']
,['CF__Cash from Divestitures','Денежные средства от отчуждения']
,['CF__Cash for Acq of Subs','Денежные средства на приобретение дочерних предприятий']
,['CF__Cash for JVs','Денежные средства для СП']
,['CF__Other Investing Activities','Прочая инвестиционная деятельность']
,['CF__Cash from Investing Activities','Денежные средства от инвестиционной деятельности']
,['CF__Dividends Paid','Выплаченные дивиденды']
,['CF__Cash From (Repayment) Debt','Денежные средства от (погашения) долга']
,['CF__Cash From (Repay) ST Debt','Денежные средства от (погашения) долга СТ']
,['CF__Cash From LT Debt','Денежные средства от долга LT']
,['CF__Repayments of LT Debt','Погашение долгосрочного долга']
,['CF__Cash (Repurchase) of Equity','Денежные средства (выкуп) собственного капитала']
,['CF__Other Financing Activities','Прочая финансовая деятельность']
,['CF__Cash from Financing Activities','Денежные средства от финансовой деятельности']
,['CF__Effect of Foreign Exchanges','Эффект от обмена валют']
,['CF__Net Changes in Cash','Чистые изменения в денежных средствах']
,['CF__Free Cash Flow','Свободный денежный поток']
,['CF__Free Cash Flow to Firm','Свободный денежный поток на фирму']
,['CF__Free Cash Flow to Equity','Свободный денежный поток на собственный капитал']
,['CF__Cash Paid for Taxes','Денежные средства, уплаченные за налоги']
,['CF__Cash Paid for Interest','Денежные средства, уплаченные за проценты']
,['CF__Cash Paid for Preferred Div','Денежные средства, уплаченные за привилегированные дивиденды']

]

DEV = pd.DataFrame([])
for vara in l___names : 
    
    ___path___ = "C:\\Users\\ivanmipt\\Dropbox\\Мой ПК (DESKTOP-RKBQ5DN)\\Desktop\\Science\\u___Universities\\u__2__1_RUS__MSU___CMF_2021_2022\\u__2__1__проект_Прогнозирование_отчётностей_и_вероятностей_дефолта___bologov\oil_gas_train\\"+vara+".xlsx"
    
    tmp = pd.read_excel(___path___, sheet_name = vara, encoding='utf-8') 
    tmp.drop_duplicates(inplace=True)
    tmp = tmp.transpose()
    
    tmp.columns = tmp.iloc[0]
    tmp = tmp[1:].reset_index(drop=True)
    
    tmp['report_date'] = pd.to_datetime(tmp['date'],format = "%Y-%m-%d")
    tmp['report_date'] = tmp['report_date'].map(lambda x: x+relativedelta(days=1))
    tmp['ticker'] = vara
    tmp = tmp[[x[0] for x in l___components]]

    print(' N : ',vara)
    
    DEV = DEV.append(tmp,ignore_index=True)


print( DEV.shape[0])

DEV1 = DEV.copy()

for vara in list(DEV1.loc[:, 'PL__Revenue':'CF__Cash Paid for Preferred Div'].columns):
    
    DEV1[vara] = DEV1[vara].map(lambda x : abs(float(x)))
    

path_ind = "C:\\Users\\ivanmipt\\Dropbox\\Мой ПК (DESKTOP-RKBQ5DN)\\Desktop\\Science\\u___Universities\\u__2__1_RUS__MSU___CMF_2021_2022\\u__2__1__проект_Прогнозирование_отчётностей_и_вероятностей_дефолта___bologov\\u__2__1__MAIN_Прогнозирование_отчётностей.xlsx"

industry = pd.read_excel(path_ind,sheet_name="Sheet1")


DEV2 = DEV1.merge(industry[['ticker','name','industry']],how='left',on=['ticker'])

print('\nNaNs : ',DEV2['industry'].isnull().sum())
    
    
path_t = "C:\\Users\\ivanmipt\\Dropbox\\Мой ПК (DESKTOP-RKBQ5DN)\\Desktop\\Science\\u___Universities\\u__2__1_RUS__MSU___CMF_2021_2022\\u__2__1__проект_Прогнозирование_отчётностей_и_вероятностей_дефолта___bologov\\DEV_financials.xlsx"
writer_t = pd.ExcelWriter(path_t,engine='xlsxwriter')

DEV2.to_excel(writer_t,sheet_name='financials')
pd.DataFrame(l___components,columns = ['строка','описание']).to_excel(writer_t,sheet_name='описание')

writer_t.save()
writer_t.close()

    

 N :  AMPY
 N :  APA
 N :  AR
 N :  BATL
 N :  BCEI
 N :  BRY
 N :  BSM
 N :  CDEV
 N :  CHK
 N :  CLR
 N :  CNX
 N :  COG
 N :  COP
 N :  CPE
 N :  CRBO
 N :  CRC
 N :  CRK
 N :  DEN
 N :  DVN
 N :  EGY
 N :  EOG
 N :  EQT
 N :  ESTE
 N :  FANG
 N :  GDP
 N :  HES
 N :  LONE
 N :  LPI
 N :  MCF
 N :  MGY
 N :  MNRL
 N :  MRO
 N :  MTDR
 N :  MUR
 N :  NOG
 N :  OAS
 N :  OVV
 N :  PDCE
 N :  PVAC
 N :  PXD
 N :  REI
 N :  RRC
 N :  SBOW
 N :  SD
 N :  SM
 N :  SWN
 N :  TALO
 N :  TPL
 N :  VNOM
 N :  WLL
 N :  WTI
 N :  XEC
 N :  XOG
2018

NaNs :  0
Wall time: 44.9 s


<br>

## Макроэкономические данные <a name=upld_macro></a>

- [Содержание](#content)

In [35]:
path_m = "C:\\Users\\ivanmipt\\Dropbox\\Мой ПК (DESKTOP-RKBQ5DN)\\Desktop\\Science\\u___Universities\\u__2__1_RUS__MSU___CMF_2021_2022\\u__2__1__проект_Прогнозирование_отчётностей_и_вероятностей_дефолта___bologov\\dataset_macro.csv"
macro = pd.read_csv(path_m,sep=",",encoding="utf-8")

macro['report_date'] = 

macro.shape[0]

42

<br>

# Feature Engineering <a name=features></a>

- [Содержание](#content)


In [169]:
reg = LinearRegression()

TRAIN = TRAIN_TEST[['var3','var4','var7','var8']] #.values.reshape(-1,1)
y = TRAIN_TEST['target']

reg.fit(TRAIN,y)

pred = reg.predict(TRAIN)
pred_  = pd.DataFrame(pred)

In [170]:
c = reg.coef_

for j in range(0,len(c)):
    print('koef ',j+1,' : ',round(c[j],2),'\n')

cc = np.corrcoef(pred,y)[0,1]
    
sd = Somers_D(y,pred)

print("cc: ",cc,"  somD: ",sd)

koef  1  :  0.06 

koef  2  :  -0.34 

koef  3  :  4.37 

koef  4  :  134.3 

cc:  0.7454423243890042   somD:  0.5616326530612245


In [ ]:
for_h.plot(x='Rating',y='distr',kind='bar',color='b')
for_h.PD.plot(secondary_y = True, linestyle='-',marker='o',color='r')

plt.savefig('D:\\Shepitko-IS\\Desktop\\Projects\\b5___Sub_Souvereigns\\Data\\b5___final_distribution.png')



In [ ]:
y = TRAIN_fin['PD']
X = TRAIN_fin.drop('PD',axis=1)

In [ ]:
logistic_regression_model = LogisticRegression(penalty='l1',C=0.75,solver='saga',multi_class='multinomial')

log_reg_model = logistic_regression_model.fit(X,y);
print('Коэффициенты наклона бета =',log_reg_model.coef_[0,:])

print('----------------------------------')
Norm_vesa_log_reg_reg = abs(log_reg_model.coef_)/nm.repmat(sum(abs(log_reg_model.coef_[0,:])),1,np.size(log_reg_model.coef_))
print("Нормированные веса для модели логистической регрессии с регуляризацией =",Norm_vesa_log_reg_reg[0,:])

print('----------------------------------')
print('Коэффициент альфа =',log_reg_model.intercept_[0])